<h3>Tesco Share Price Chart</h3>

<h4><p>A quick script to plot the Tesco Share price from Yahoo finance and calculate the weekly rolling average.
    
I'm going to use pandas-datareader and will plot with Bokeh</h4>
 
<p>
First Import Panda Datareader and DateTime

In [1]:
from pandas_datareader import data
import datetime

<p>
Setting the time range for the graph and then importing the data from yahoo using datareader

In [4]:
end = datetime.datetime.now().date()-datetime.timedelta(1)
start = datetime.datetime.now().date()-datetime.timedelta(335)
df = data.DataReader(name='TSCO',data_source="yahoo",start= start ,end= end)

<p>
Create a function (stock_change) to evaluate where one input value (c) is an increase, decrease or no change from a second inout value (o)

In [5]:
def stock_change(c,o):
    if(c > o):
        return "increase"
    elif(c< o):
        return "decrease"
    else:
        return "equal"



Add three new columns to the dataframe (df):

Status - use function stock_change to see how the close price 
compares to the open price

Middle - is the mid point between Open and Close

Height - is the difference between Open and Close in absolute terms

7-day rolling averge - calculate average over the last 7 days (5 working days) from Close price

In [17]:
df["Status"]=[stock_change(c,o) for c,o in zip(df.Close,df.Open)]
df["Middle"]=(df.Open+df.Close)/2
df["Height"]=abs(df.Open-df.Close)

df["Rolling_Average"]= df['Close'].rolling(5).mean()

df

,High,Low,Open,Close,Volume,Adj Close,Status,Middle,Height,7-day Rolling Average,7_day_Rolling_Average,Rolling_Average
Date,,,,,,,,,,,,
2019-08-14,102.889999,99.760002,102.209999,99.839996,1241500.0,98.467972,decrease,101.024998,2.370003,NaN,NaN,NaN
2019-08-15,100.779999,98.599998,99.989998,99.360001,1371500.0,97.994576,decrease,99.674999,0.629997,NaN,NaN,NaN
2019-08-16,101.250000,99.699997,100.129997,100.550003,1078600.0,99.168236,increase,100.340000,0.420006,NaN,NaN,NaN
2019-08-19,102.120003,99.669998,101.750000,101.209999,1375100.0,99.819160,decrease,101.480000,0.540001,NaN,NaN,NaN
2019-08-20,102.370003,100.529999,100.919998,100.870003,865900.0,99.483833,decrease,100.895000,0.049995,100.366000,100.366000,100.366000
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-07,135.429993,132.899994,133.179993,134.229996,823400.0,134.229996,increase,133.704994,1.050003,132.981995,132.981995,132.981995
2020-07-08,135.699997,132.960007,134.149994,135.610001,734600.0,135.610001,increase,134.879997,1.460007,133.745996,133.745996,133.745996
2020-07-09,138.059998,135.160004,136.500000,137.699997,976500.0,137.699997,increase,137.099998,1.199997,134.749997,134.749997,134.749997


Plot data using bokeh

The figure is a candelstick chart saved as cs.html

First plot the high-to-low values as black line (p.segment)

Next, plot rectangles for showing the range between the open and close prices (green for days of stock gain, red for loses)

In [7]:
from bokeh.plotting import figure,show, output_file

In [20]:
p=figure(x_axis_type='datetime', title="Candelstick Chart for the Tesco Share Price", width=600,height=300)
p.grid.grid_line_alpha=0.3

p.segment(df.index,df.High,df.index,df.Low,line_color='black')

p.rect(df.index[df.Status=="increase"],
       df.Middle[df.Status=="increase"],
       (12*60*60*1000),
       df.Height[df.Status=="increase"],
        fill_color='green')

p.rect(df.index[df.Status=="decrease"],
       df.Middle[df.Status=="decrease"],
       (12*60*60*1000),
       df.Height[df.Status=="decrease"],
        fill_color='red')

p.line(df.index,df.Rolling_Average)

output_file('cs.html')
show(p)

The following code enables the output file (cs.html) to be viewed here in jypter notebook. The zoom and other functions on the RHS will work

In [26]:
from IPython.display import IFrame

IFrame(src='cs.html', width=700, height=600)

Also, create the graph as an embeded graph in a website

In [27]:
from bokeh.embed import components
from bokeh.resources  import CDN

In [28]:
java_script,web =components(p)
cdn_js=CDN.js_files


the output embedded graph can be seen on the webpage below;

https://bourne123.herokuapp.com/plot/